# Retrogate Analysis

1: über alle 3-3 Zustände loopen

2: beste Lösung berechnen, abspeichern

2: immer die Lösung, mit besten berechneten Wert abspeichern

3: diese Lösung auch nach Anwendung aller Symmetrieoptionen und Spielersteinwechsel speichern


In [ ]:
%run ./nmm-game-utils.ipynb
%run ./nmm-symmetry.ipynb

In [ ]:
startBoard = ((' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '))

def generateBoards(board=startBoard, x=0, y=0):
    _boards  = []
    white = countStones(('', board), 'w')
    black = countStones(('', board), 'b')
    if (white == 3 and black == 3):
        return tuple([board])
    for i in range(x,3):
        for j in range(y,8):
            if board[i][j] == ' ':
                if white == 3:
                    newBoard = place(board, (i, j), 'b')
                    _boards += (generateBoards(newBoard, i, j))
                elif black == 3:
                    newBoard = place(board, (i, j), 'w')
                    _boards += generateBoards(newBoard, i, j)
                else:
                    for turn in ('w', 'b'):
                        newBoard = place(board, (i, j), turn)
                        _boards += generateBoards(newBoard, i, j)
    return _boards

In [ ]:
def getSymmetryUnique():
    boards = generateBoards()
    doubleBoards = set()
    uniqueBoards = []
    for board in boards:
        state = tuple(['', board])
        if not (state in doubleBoards):
            doubleBoards |= findSymmetries(('', board))
            uniqueBoards.append(board)
    return uniqueBoards